In [69]:
import numpy as np

class Linear:
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.weights = np.random.randn(in_features, out_features) * 0.01
        self.biases = np.zeros((1, out_features))
        self.input = None
        self.grad_weights = None
        self.grad_biases = None

    def forward(self, x):
        self.input = x
        return np.dot(x, self.weights) + self.biases

    def backward(self, d_out):
        self.grad_weights = np.dot(self.input.T, d_out)
        self.grad_biases = np.sum(d_out, axis=0, keepdims=True)
        d_input = np.dot(d_out, self.weights.T)
        return d_input


In [70]:
class ReLU:
    def forward(self, x):
        self.input = x
        return np.maximum(0, x)

    def backward(self, d_out):
        d_input = d_out.copy()
        d_input[self.input <= 0] = 0
        return d_input


In [71]:
class Sigmoid:
    def forward(self, x):
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, d_out):
        return d_out * (self.output * (1 - self.output))


In [72]:
class Tanh:
    def forward(self, x):
        self.output = np.tanh(x)
        return self.output

    def backward(self, d_out):
        return d_out * (1 - self.output ** 2)


In [73]:
class Softmax:
    def forward(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output = exps / np.sum(exps, axis=1, keepdims=True)
        return self.output

    def backward(self, d_out):
        # Assuming d_out is already the gradient of loss w.r.t. softmax output
        return d_out


In [74]:
class CrossEntropyLoss:
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        self.n = y_true.shape[0]
        loss = -np.sum(y_true * np.log(y_pred + 1e-10)) / self.n
        return loss

    def backward(self):
        return (self.y_pred - self.y_true) / self.y_true.shape[0]


In [75]:
class MSELoss:
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return np.mean((y_pred - y_true) ** 2)

    def backward(self):
        return 2 * (self.y_pred - self.y_true) / self.y_true.size


In [76]:
class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def step(self, layer):
        if hasattr(layer, 'grad_weights'):
            layer.weights -= self.learning_rate * layer.grad_weights
            layer.biases -= self.learning_rate * layer.grad_biases


In [77]:
class Model:
    def __init__(self):
        self.layers = []
        self.loss_fn = None
        self.optimizer = None

    def add_layer(self, layer):
        self.layers.append(layer)

    def compile(self, loss_fn, optimizer):
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, loss_grad):
        for layer in reversed(self.layers):
            loss_grad = layer.backward(loss_grad)

    def train(self, x_train, y_train, epochs, batch_size):
        for epoch in range(epochs):
            # Shuffle training data
            indices = np.arange(x_train.shape[0])
            np.random.shuffle(indices)
            x_train, y_train = x_train[indices], y_train[indices]
            
            # Mini-batch training
            for start in range(0, x_train.shape[0], batch_size):
                end = min(start + batch_size, x_train.shape[0])
                x_batch, y_batch = x_train[start:end], y_train[start:end]
                
                # Forward pass
                predictions = self.forward(x_batch)
                
                # Compute loss
                loss = self.loss_fn.forward(predictions, y_batch)
                
                # Backward pass
                loss_grad = self.loss_fn.backward()
                self.backward(loss_grad)
                
                # Update parameters
                for layer in self.layers:
                    self.optimizer.step(layer)
                
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss}')

    def evaluate(self, x_test, y_test):
        predictions = self.forward(x_test)
        loss = self.loss_fn.forward(predictions, y_test)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
        return loss, accuracy

    def save(self, filename):
        pass

    def load(self, filename):
        pass


In [78]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

# Load dataset
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

# Separate features and labels
x = train_df.drop(columns='label').values
y = train_df['label'].values

# Normalize pixel values
x = x / 255.0

# Reshape data to (num_samples, 28, 28)
x = x.reshape(-1, 28, 28)

# One-hot encode the labels
y = to_categorical(y, 10)

# Flatten the data
x = x.reshape(x.shape[0], -1)

# Split data into training and validation sets

np.random.seed(42)  
indices = np.arange(x.shape[0])
np.random.shuffle(indices)
split_index = int(0.8 * len(indices))  # 80% for training, 20% for validation

train_indices = indices[:split_index]
val_indices = indices[split_index:]

x_train, x_val = x[train_indices], x[val_indices]
y_train, y_val = y[train_indices], y[val_indices]


In [79]:
# Initialize model and layers
model = Model()
model.add_layer(Linear(784, 128))
model.add_layer(ReLU())
model.add_layer(Linear(128, 10))
model.add_layer(Softmax())

loss_fn = CrossEntropyLoss()
optimizer = SGD(learning_rate=0.7)
model.compile(loss_fn, optimizer)

model.train(x_train, y_train, epochs=20, batch_size=64)

val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


Epoch 1/20, Loss: 0.20843550992723342
Epoch 2/20, Loss: 0.08247531022362248
Epoch 3/20, Loss: 0.26419405883581965
Epoch 4/20, Loss: 0.023669446127142335
Epoch 5/20, Loss: 0.10499128563794523
Epoch 6/20, Loss: 0.0583920196586511
Epoch 7/20, Loss: 0.020803659266510476
Epoch 8/20, Loss: 0.005990714559860159
Epoch 9/20, Loss: 0.006563529149793313
Epoch 10/20, Loss: 0.01015675957008653
Epoch 11/20, Loss: 0.015729013246983305
Epoch 12/20, Loss: 0.004346433137511537
Epoch 13/20, Loss: 0.005053163384984366
Epoch 14/20, Loss: 0.003937922445715305
Epoch 15/20, Loss: 0.00256072955598431
Epoch 16/20, Loss: 0.0023850757817414775
Epoch 17/20, Loss: 0.0014737783023120628
Epoch 18/20, Loss: 0.0006843251203996258
Epoch 19/20, Loss: 0.0002722352386597173
Epoch 20/20, Loss: 0.0005156941904956196
Validation Loss: 0.10205652933361634
Validation Accuracy: 0.9772619047619048
